In [64]:
import numpy as np
import pandas as pd

Problem: Given features predict income

In [65]:
#description states that missing values in this data set are denoted with a '?'; so parse NaN = ?
#since theres no header in the dataset itself, it shouldnt be generated from one of the rows

adult_dataframe = pd.read_csv("data/complete-adult.csv",na_values='?',header=None)
adult_dataframe.head()


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [66]:
adult_dataframe.shape
cols = ["age","work class","final weight","education ","education years","marital status","occupation","relationship in family","race","sex","capital gain","capital loss","weekly hours","native country","income"]
adult_dataframe.columns = cols

In [67]:
#sources of dirty values: it contain lot's of outlier, missing value, or others intentionly wrong filling. 
#dealing with missing values:
adult_dataframe.isna().sum()
# no_of_na/adult_dataframe.shape[0]*100


age                          0
work class                2799
final weight                 0
education                    0
education years              0
marital status               0
occupation                2809
relationship in family       0
race                         0
sex                          0
capital gain                 0
capital loss                 0
weekly hours                 0
native country             857
income                       0
dtype: int64

In [68]:
#since its not a lot of rows with unknown values, we'll drop them for now, but will keep in mind that the work class and occupation unknowns seem to be related? also should be remove nan or replace with average/most_frequent?

adult_df_nadrop = adult_dataframe.dropna() #this doesnt change the dataframe directly - so it has so be saved to a variable 
adult_df_nadrop.shape

(45222, 15)

In [69]:
adult_df_nadrop.head()
#maybe we can change income to a boolean where its true if income is greater than 50k(an attempt to simplify data)

,age,work class,final weight,education,education years,marital status,occupation,relationship in family,race,sex,capital gain,capital loss,weekly hours,native country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [70]:
#description also says that theres people without pay and never worked, and sincd they have (almost) no income, although theyd be classified under 50k, thats not very meaningful
#check outliers
#combine categories 
#convert categories to numbers?
#normalize other numerical data